In [13]:
import os

dataset_dir =  '/Users/y_tamura/Downloads/all'
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

train = os.listdir(train_dir)
test = os.listdir(test_dir)

In [ ]:
#trainデータの８割を学習用、残りをバリデーション用として使用する


In [6]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='softmax'))

Using TensorFlow backend.


In [7]:
from keras import optimizers

model.compile(loss='binary_crossentropy', 
                             optimizer= optimizers.Adam(lr=1e-4), 
                             metrics= ['acc'])

In [8]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callback_cp = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5')

callback_es = EarlyStopping(monitor='val_acc', patience=2, mode='auto', verbose=1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                                train_dir, 
                                target_size=(150, 150),
                                batch_size=32, 
                                class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
                                            validation_dir, 
                                            target_size=(150,150), 
                                            batch_size=32, 
                                            class_mode='binary')